In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read in data
df = pd.read_csv('C:\Documents\Premi_ClubQ8.csv', sep = ';', decimal=",")

#split date and time
df[['REQUEST_DATE','REQUEST_TIME']] = df.DATA_OPERAZIONE.str.split(" ",expand=True,)
#drop the original column
df = df.drop(columns="DATA_OPERAZIONE")
#rename columns
df.columns = ['CUSTOMER_CODE', 'REQUEST_PLACE', 'CATEGORY', 'MATERIAL_GROUPING', 'PRIZE_DESCRIPTION', 'POINTS_REQUIRED', 'CUSTOMER_CONTRIBUTION', 'DATE', 'TIME']
#convert contribution to float
df['CUSTOMER_CONTRIBUTION'] = df['CUSTOMER_CONTRIBUTION'].astype(float)

df

,CUSTOMER_CODE,REQUEST_PLACE,CATEGORY,MATERIAL_GROUPING,PRIZE_DESCRIPTION,POINTS_REQUIRED,CUSTOMER_CONTRIBUTION,DATE,TIME
0,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:16:21
1,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:17:30
2,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:18:49
3,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:19:42
4,7071161603014051338,PUNTO VENDITA,PREMIO FISICO,PER TE,PRENOTAZIONE PORTA CARTE CREDITO SPARCO,450.0,0.0,2020-09-01,00:34:24
...,...,...,...,...,...,...,...,...,...
501444,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRODOMESTICI,PRENOTAZIONE PHILIPS RASOIO,2395.0,0.0,2021-08-31,23:14:00
501445,7071161603670271808,PUNTO VENDITA,PREMIO FISICO,MODA,PRENOTAZIONE OROLOGIO SECTOR,3500.0,30.0,2021-08-31,23:14:59
501446,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRONICA,PRENOTAZIONE AVVITATORE,4200.0,15.0,2021-08-31,23:15:02
501447,7071161603007816325,PUNTO VENDITA,PREMIO FISICO,CASA,PRENOTAZIONE PORTABIANCHERIA,4175.0,0.0,2021-08-31,23:24:29


The dataset has 501449 rows and 9 columns. 

In [3]:
#define a function that counts the number of na's and gives also the percentage of total observations in a column that is na
def summarize_na(df: pd.DataFrame) -> pd.DataFrame:
    nan_count = df.isna().sum()
    return pd.DataFrame({'nan_count': nan_count, 
                         'nan_pct': nan_count / len(df) * 100
                         }
                        )[nan_count > 0]

In [4]:
#look at the number of na's for different columns
df_nan_sum = summarize_na(df)
df_nan_sum

,nan_count,nan_pct
POINTS_REQUIRED,229,0.045668
CUSTOMER_CONTRIBUTION,37895,7.557100


In [5]:
#finding the unique levels of the factor variable category
df['CATEGORY'].unique()

array(['PREMIO FISICO', 'BUONO SCONTO CARBURANTE'], dtype=object)

In [24]:
def get_frequencies(series: pd.Series, n_categories: int = None, 
                    bins: int = None, dropna: bool = True
                    ) -> pd.DataFrame:
    """Return a DataFrame displaying the series frequencies.
    
    Parameters
    ----------
    series: pd.Series
        The series for which frequencies are to be computed.
    n_categories: int, optional
        Maximum number of categories to return in output.
        Low-frequency categories will be grouped together to
        reach this threshold.
        
    """
    vc = series.value_counts(ascending=False, 
                             bins=bins, 
                             dropna=dropna
                             )
    if n_categories is not None:
        if not isinstance(n_categories, int) or n_categories <= 0:
            raise TypeError('n_categories should be a strictly positive integer')
        if n_categories < len(vc):
            freq_others = vc.iloc[n_categories - 1:].sum()
            vc = vc.iloc[:n_categories - 1]\
                   .append(pd.Series({'others': freq_others}))
    return pd.DataFrame({'absolute': vc, 
                         'relative': vc / len(series),
                         },
                        index=vc.index
                        )

In [25]:
# what is the proportion of transactions for prizes that has been done online compared to physcial
get_frequencies(df['REQUEST_PLACE'], n_categories=5)

,absolute,relative
PUNTO VENDITA,469059,0.935407
ONLINE,32390,0.064593


In [26]:
#this shows the difference in type of prize that customers obtained
get_frequencies(df['CATEGORY'], n_categories=5)

,absolute,relative
BUONO SCONTO CARBURANTE,272660,0.543744
PREMIO FISICO,228789,0.456256


In [27]:
#make an overview of the different groups of prizes and the proportion of occurence
get_frequencies(df['MATERIAL_GROUPING'], n_categories=20)

,absolute,relative
BUONO SCONTO CARBURANTE,272660,0.543744
PER LA TUA CASA,68450,0.136504
PER TE,30569,0.060961
ELETTRONICA,24924,0.049704
TAVOLA & CUCINA,20407,0.040696
CASA,18529,0.036951
ELETTRODOMESTICI,17489,0.034877
PER IL TUO BENESSERE,14858,0.029630
TEMPO LIBERO,9569,0.019083
MODA,7237,0.014432


In [28]:

#overview of all the awarded prizes in the dataset along with proportion of total that they got awarded
get_frequencies(df['PRIZE_DESCRIPTION'], n_categories=5)

,absolute,relative
BUONO SCONTO CARB 4E 430 PUNTI,88550,0.176588
BUONO SCONTO CARB 4E 860 PUNTI,70302,0.140198
BUONO SCONTO CARB 4E 1290 PUNTI,66117,0.131852
BUONO SCONTO CARB 4E 2580 PUNTI,47553,0.094831
others,228927,0.456531


In [29]:
# proportions of different levels of required points in the transaction
get_frequencies(df['POINTS_REQUIRED'], n_categories=5)

,absolute,relative
430.0,272660,0.543744
650.0,48678,0.097075
350.0,19034,0.037958
390.0,11792,0.023516
others,149056,0.297251


In [30]:
#proportion of different contribution levels
get_frequencies(df['CUSTOMER_CONTRIBUTION'], n_categories=5)

,absolute,relative
0.0,389550,0.776849
5.0,13449,0.026820
2.0,7262,0.014482
8.0,6811,0.013583
others,46482,0.092695


In [31]:
#proportion of transactions of the total that are performed on each day
get_frequencies(df['DATE'], n_categories=5)

,absolute,relative
2021-03-15,2765,0.005514
2020-09-30,2158,0.004304
2021-03-31,2150,0.004288
2021-03-26,2129,0.004246
others,492247,0.981649


In [13]:
#we read in the customer database
df2 = pd.read_csv('C:\Documents\Anagrafica_ClubQ8.csv', sep = ';', decimal=",")

#copy the date column
df2['DATE_JOINED'] = df2['DATA_BATTESIMO']
#drop the original
df2 = df2.drop(columns="DATA_BATTESIMO")
#rename all the columns
df2.columns = ['CUSTOMER_CODE', 'SEX', 'DOB', 'REGION', 'PROVINCE', 'CITY', 'TYPE_OF_CARD' ,'TOTAL_POINTS', 'DATE_JOINED']

df2

,CUSTOMER_CODE,SEX,DOB,REGION,PROVINCE,CITY,TYPE_OF_CARD,TOTAL_POINTS,DATE_JOINED
0,7071161603018922195,M,1951-09-14 00:00:00,LAZIO,VITERBO,SUTRI,STARQ8_FISICA,511.0,2015-01-08 12:32:58
1,7071161603005992565,F,1988-09-15 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,379.0,2020-07-10 12:22:35
2,7071161603025867243,F,1943-09-09 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,108.0,2020-07-10 18:07:14
3,7071161603021997861,M,1974-03-01 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,100.0,2020-07-10 18:08:21
4,7071161603025554049,M,1963-11-22 00:00:00,CAMPANIA,SALERNO,EBOLI,STARQ8_FISICA,284.0,2020-07-10 18:09:27
...,...,...,...,...,...,...,...,...,...
1656132,7071161603672630225,F,1962-06-22 00:00:00,EMILIA ROMAGNA,RIMINI,SAN GIOVANNI IN MARIGNANO,STARQ8_FISICA,2741.0,2019-11-27 18:06:31
1656133,7071161603673015798,M,1967-03-19 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,8996.0,2019-03-21 08:34:30
1656134,7071161603024035909,M,1987-03-18 00:00:00,SICILIA,PALERMO,PALERMO,STARQ8_FISICA,424.0,2021-01-19 09:07:37
1656135,7071161603601002132,F,1984-09-20 00:00:00,LOMBARDIA,BRESCIA,ISEO,STARQ8_VIRTUALE,740.0,2020-12-15 22:57:21


In [15]:
#merge the prizes database with the customer database
df = pd.merge(df, df2, how = 'left')
df

,CUSTOMER_CODE,REQUEST_PLACE,CATEGORY,MATERIAL_GROUPING,PRIZE_DESCRIPTION,POINTS_REQUIRED,CUSTOMER_CONTRIBUTION,DATE,TIME,SEX,DOB,REGION,PROVINCE,CITY,TYPE_OF_CARD,TOTAL_POINTS,DATE_JOINED
0,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:16:21,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
1,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:17:30,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
2,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:18:49,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
3,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:19:42,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
4,7071161603014051338,PUNTO VENDITA,PREMIO FISICO,PER TE,PRENOTAZIONE PORTA CARTE CREDITO SPARCO,450.0,0.0,2020-09-01,00:34:24,M,1967-09-02 00:00:00,SICILIA,PALERMO,CEFALU',STARQ8_FISICA,5279.0,2001-10-05 11:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501444,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRODOMESTICI,PRENOTAZIONE PHILIPS RASOIO,2395.0,0.0,2021-08-31,23:14:00,M,1975-07-11 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,111.0,2017-07-04 13:00:40
501445,7071161603670271808,PUNTO VENDITA,PREMIO FISICO,MODA,PRENOTAZIONE OROLOGIO SECTOR,3500.0,30.0,2021-08-31,23:14:59,M,1977-11-29 00:00:00,CAMPANIA,NAPOLI,NAPOLI,STARQ8_FISICA,9590.0,2012-11-08 10:28:34
501446,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRONICA,PRENOTAZIONE AVVITATORE,4200.0,15.0,2021-08-31,23:15:02,M,1975-07-11 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,111.0,2017-07-04 13:00:40
501447,7071161603007816325,PUNTO VENDITA,PREMIO FISICO,CASA,PRENOTAZIONE PORTABIANCHERIA,4175.0,0.0,2021-08-31,23:24:29,M,1967-11-13 00:00:00,MARCHE,ANCONA,ANCONA,STARQ8_FISICA,1041.0,2015-08-28 15:56:15
